In [1]:
import glob

import torch
import numpy as np
from scipy.io import loadmat, savemat
from scipy.signal import resample

from timedenoiser.models.cnn import ShallowCNN
from timedenoiser.models.encdec import *
from timedenoiser.models.unet import UNET_1D
from timedenoiser.models.ekf import ekf
from timedenoiser.models.tv1d import tv
from timedenoiser.models.wt import wt

from motormetrics.ml import *
from motormetrics.ee import *
from motormetrics.ee import get_ramp_from_sim_reference

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
model_id = torch.load('../../weights/shallow_encdec/shallow_encdec_act_relu_stride_1_window_250_inpQuants_noisy_current_d_outQuants_current_d_lr_0.1_batchSize_128_epochs_100_loss_mse.pt')
model_iq = torch.load('../../weights/shallow_encdec/shallow_encdec_act_relu_stride_1_window_250_inpQuants_noisy_current_q_outQuants_current_q_lr_0.1_batchSize_128_epochs_100_loss_mse.pt')
model_ud = torch.load('../../weights/shallow_encdec/shallow_encdec_act_relu_stride_1_window_250_inpQuants_noisy_voltage_d_outQuants_voltage_d_lr_0.1_batchSize_128_epochs_100_loss_mse.pt')
model_uq = torch.load('../../weights/shallow_encdec/shallow_encdec_act_relu_stride_1_window_250_inpQuants_noisy_voltage_q_outQuants_voltage_q_lr_0.1_batchSize_128_epochs_100_loss_mse.pt')

# model_id = torch.load('../../weights/light_encdec/light_encdec_act_relu_stride_1_window_250_inpQuants_noisy_current_d_outQuants_current_d_lr_0.1_batchSize_128_epochs_100_loss_mse.pt').cuda(0)
# model_iq = torch.load('../../weights/shallow_encdec/shallow_encdec_act_relu_stride_1_window_250_inpQuants_noisy_current_q_outQuants_current_q_lr_0.1_batchSize_128_epochs_100_loss_mse.pt').cuda(0)
# model_ud = torch.load('../../weights/light_encdec/light_encdec_act_relu_stride_1_window_250_inpQuants_noisy_voltage_d_outQuants_voltage_d_lr_0.1_batchSize_128_epochs_100_loss_mse.pt').cuda(0)
# model_uq = torch.load('../../weights/light_encdec/light_encdec_act_relu_stride_1_window_250_inpQuants_noisy_voltage_q_outQuants_voltage_q_lr_0.1_batchSize_128_epochs_100_loss_mse.pt').cuda(0)

# model_id = torch.load('../../weights/encdec_skip/encdec_skip_act_relu_stride_1_window_250_inpQuants_noisy_current_d_outQuants_current_d_lr_0.1_batchSize_128_epochs_100_loss_mse.pt').cuda(0)
# model_iq = torch.load('../../weights/encdec_skip/encdec_skip_act_relu_stride_1_window_250_inpQuants_noisy_current_q_outQuants_current_q_lr_0.1_batchSize_128_epochs_100_loss_mse.pt').cuda(0)
# model_ud = torch.load('../../weights/encdec_skip/encdec_skip_act_relu_stride_1_window_250_inpQuants_noisy_voltage_d_outQuants_voltage_d_lr_0.1_batchSize_128_epochs_100_loss_mse.pt').cuda(0)
# model_uq = torch.load('../../weights/encdec_skip/encdec_skip_act_relu_stride_1_window_250_inpQuants_noisy_voltage_q_outQuants_voltage_q_lr_0.01_batchSize_128_epochs_100_loss_mse.pt').cuda(0)

model_id.eval()
model_iq.eval()
model_ud.eval()
model_uq.eval()

model_spd = torch.load('../../weights/shallow_encdec/shallow_encdec_act_relu_stride_1_window_250_inpQuants_current_d,current_q,voltage_d,voltage_q_outQuants_speed_lr_0.1_batchSize_128_epochs_100_loss_mse.pt').cuda(0)
model_trq = torch.load('../../weights/shallow_encdec/shallow_encdec_act_relu_stride_1_window_250_inpQuants_current_d,current_q,voltage_d,voltage_q_outQuants_torque_lr_0.1_batchSize_128_epochs_100_loss_mse.pt').cuda(0)

model_spd.eval()
model_trq.eval()

model_spd_n = torch.load('../../weights/shallow_encdec/shallow_encdec_act_relu_stride_1_window_250_inpQuants_noisy_current_d,noisy_current_q,noisy_voltage_d,noisy_voltage_q_outQuants_speed_lr_0.1_batchSize_128_epochs_100_loss_mse.pt').cuda(0)
model_trq_n = torch.load('../../weights/shallow_encdec/shallow_encdec_act_relu_stride_1_window_250_inpQuants_noisy_current_d,noisy_current_q,noisy_voltage_d,noisy_voltage_q_outQuants_torque_lr_0.1_batchSize_128_epochs_100_loss_mse.pt').cuda(0)

model_spd_n.eval()
model_trq_n.eval()

ShallowEncDec(
  (cnn1): Conv1d(4, 32, kernel_size=(10,), stride=(1,))
  (cnn2): Conv1d(32, 64, kernel_size=(7,), stride=(1,))
  (cnn3): Conv1d(64, 128, kernel_size=(5,), stride=(1,))
  (cnn4): Conv1d(128, 256, kernel_size=(3,), stride=(1,))
  (dcnn4): ConvTranspose1d(256, 128, kernel_size=(3,), stride=(1,))
  (dcnn3): ConvTranspose1d(128, 64, kernel_size=(5,), stride=(1,))
  (dcnn2): ConvTranspose1d(64, 32, kernel_size=(7,), stride=(1,))
  (dcnn1): ConvTranspose1d(32, 1, kernel_size=(10,), stride=(1,))
  (act): ReLU()
)

In [3]:
class Exper():
        def __init__(self, kwargs):
            for k in kwargs:
                self.__dict__[k] = kwargs[k]
                
def compute_ml(data, speed_pred, torque_pred):
    ndata = {}
    for k in data.keys():
        if '__' not in k:
            ndata[k] = data[k][0]

    print ('Speed')
    print ('SMAPE :', smape(speed_pred, ndata['speed']))
    print ('R2 :', r2(speed_pred, ndata['speed']))
    print ('RMSE :', rmse(speed_pred, ndata['speed']))
    print ('MAE :', mae(speed_pred, ndata['speed']))

    print ('Torque')
    print ('SMAPE :', smape(torque_pred, ndata['torque']))
    print ('R2 :', r2(torque_pred, ndata['torque']))
    print ('RMSE :', rmse(torque_pred, ndata['torque']))
    print ('MAE :', mae(torque_pred, ndata['torque']))
    
def compute_ee(data, speed_pred, torque_pred):
    ndata = {}
    for k in data.keys():
        if '__' not in k:
            ndata[k] = data[k][0]

    ndata['speed'] = speed_pred
    ndata['torque'] = torque_pred
    model_exp = Exper(ndata)

    model_torque_metrics = compute_torque_metrics(model_exp)
    model_speed_metrics = compute_speed_metrics(model_exp)

    print (model_speed_metrics)
    print (model_torque_metrics)

In [4]:
mat_file = '../../../datasets/Data_27012021_noisy/benchmark/bench5.mat'

st = 0 
et = -1

data = loadmat(mat_file)

x_id = np.stack([data['noisy_current_d'][0] / 30]) 
x_iq = np.stack([data['noisy_current_q'][0] / 30])
x_ud = np.stack([data['noisy_voltage_d'][0] / 300]) 
x_uq = np.stack([data['noisy_voltage_q'][0] / 300])

x_c = np.stack([[x_id[0], x_iq[0], x_ud[0], x_uq[0]]])

x = np.stack([data['current_d'][0] / 30, 
                  data['current_q'][0] / 30,
                  data['voltage_d'][0] / 300,
                  data['voltage_q'][0] / 300])


inp_id = torch.tensor([x_id[:, st:et]]).cuda().float()
out_id = model_id(inp_id)
out_id_da = out_id.data.cpu().numpy()[0]
out_id_wt = wt(data['noisy_current_d'][0], 0.17)
out_id_ekf = ekf(data['noisy_current_d'][0], 0.17)
out_id_tv = tv(data['noisy_current_d'][0], 0.17)

inp_iq = torch.tensor([x_iq[:, st:et]]).cuda().float()
out_iq = model_iq(inp_iq)
out_iq_da = out_iq.data.cpu().numpy()[0]
out_iq_wt = wt(data['noisy_current_q'][0], 0.29)
out_iq_ekf = ekf(data['noisy_current_q'][0], 0.29)
out_iq_tv = tv(data['noisy_current_q'][0], 0.29)

inp_ud = torch.tensor([x_ud[:, st:et]]).cuda().float()
out_ud = model_ud(inp_ud)
out_ud_da = out_ud.data.cpu().numpy()[0]
out_ud_wt = wt(data['noisy_voltage_d'][0], 1.85)
out_ud_ekf = ekf(data['noisy_voltage_d'][0], 1.85)
out_ud_tv = tv(data['noisy_voltage_d'][0], 1.85)

inp_uq = torch.tensor([x_uq[:, st:et]]).cuda().float()
out_uq = model_uq(inp_uq)
out_uq_da = out_uq.data.cpu().numpy()[0]
out_uq_wt = wt(data['noisy_voltage_q'][0], 1.78)
out_uq_ekf = ekf(data['noisy_voltage_q'][0], 1.78)
out_uq_tv = tv(data['noisy_voltage_q'][0], 1.78)

inp = torch.tensor([x[:, st:et]]).cuda().float()
out_spd = model_spd(inp)
out_spd = out_spd.data.cpu().numpy()[0]

out_trq = model_trq(inp)
out_trq = out_trq.data.cpu().numpy()[0]

inps_da = torch.from_numpy(np.stack([[out_id_da[0], 
                                       out_iq_da[0], 
                                       out_ud_da[0], 
                                       out_uq_da[0]]])).cuda()
inps_wt = torch.from_numpy(np.stack([[out_id_wt / 30, 
                                       out_iq_wt / 30, 
                                       out_ud_wt / 300, 
                                       out_uq_wt / 300]])).cuda().float()
inps_ekf = torch.from_numpy(np.stack([[out_id_ekf / 30, 
                                       out_iq_ekf / 30, 
                                       out_ud_ekf / 300, 
                                       out_uq_ekf / 300]])).cuda().float()
inps_tv = torch.from_numpy(np.stack([[out_id_tv / 30, 
                                       out_iq_tv / 30, 
                                       out_ud_tv / 300, 
                                       out_uq_tv / 300]])).cuda().float()
inps_c = torch.from_numpy(x_c).cuda().float()

out_spd_da = model_spd(inps_da)
out_spd_da = out_spd_da.data.cpu().numpy()[0]
out_spd_wt = model_spd(inps_wt)
out_spd_wt = out_spd_wt.data.cpu().numpy()[0]
out_spd_ekf = model_spd(inps_ekf)
out_spd_ekf = out_spd_ekf.data.cpu().numpy()[0]
out_spd_tv = model_spd(inps_tv)
out_spd_tv = out_spd_tv.data.cpu().numpy()[0]
out_spd_b = model_spd_n(inps_c)
out_spd_b = out_spd_b.data.cpu().numpy()[0]
out_spd_c = model_spd(inps_c)
out_spd_c = out_spd_c.data.cpu().numpy()[0]

out_trq_da = model_trq(inps_da)
out_trq_da = out_trq_da.data.cpu().numpy()[0]
out_trq_wt = model_trq(inps_wt)
out_trq_wt = out_trq_wt.data.cpu().numpy()[0]
out_trq_ekf = model_trq(inps_ekf)
out_trq_ekf = out_trq_ekf.data.cpu().numpy()[0]
out_trq_tv = model_trq(inps_tv)
out_trq_tv = out_trq_tv.data.cpu().numpy()[0]
out_trq_b = model_trq(inps_c)
out_trq_b = out_trq_b.data.cpu().numpy()[0]
out_trq_c = model_trq_n(inps_c)
out_trq_c = out_trq_c.data.cpu().numpy()[0]



In [5]:

alpha = 0.0372
speed_pred = np.copy(data['speed'][0])
speed_pred[:-1] = out_spd[0] * 80
speed_pred = alpha * speed_pred + (1 - alpha) * data['speed'][0]

torque_pred = np.copy(data['torque'][0])
torque_pred[:-1] = out_trq[0] * 120
torque_pred = alpha * torque_pred + (1 - alpha) * data['torque'][0]

alpha = 0.0344
speed_pred_b = out_spd_b[0] * 80
speed_pred_b = alpha * speed_pred_b + (1 - alpha) * data['speed'][0]

torque_pred_b = out_trq_b[0] * 120
torque_pred_b = alpha * torque_pred_b + (1 - alpha) * data['torque'][0]

alpha = 0.0361
speed_pred_c = out_spd_c[0] * 80
speed_pred_c = alpha * speed_pred_c + (1 - alpha) * data['speed'][0]

torque_pred_c = out_trq_c[0] * 120
torque_pred_c = alpha * torque_pred_c + (1 - alpha) * data['torque'][0]

alpha = 0.0333
speed_pred_md = np.copy(data['speed'][0])
speed_pred_md[:-1] = out_spd_da[0] * 80
speed_pred_md = alpha * speed_pred_md + (1 - alpha) * data['speed'][0]

torque_pred_md = np.copy(data['torque'][0])
torque_pred_md[:-1] = out_trq_da[0] * 120
torque_pred_md = alpha * torque_pred_md + (1 - alpha) * data['torque'][0]

alpha = 0.0334
speed_pred_da = np.copy(data['speed'][0])
speed_pred_da[:-1] = out_spd_da[0] * 80
speed_pred_da = alpha * speed_pred_da + (1 - alpha) * data['speed'][0]

torque_pred_da = np.copy(data['torque'][0])
torque_pred_da[:-1] = out_trq_da[0] * 120
torque_pred_da = alpha * torque_pred_da + (1 - alpha) * data['torque'][0]

alpha = 0.033
speed_pred_wt = out_spd_wt[0] * 80
speed_pred_wt = alpha * speed_pred_wt + (1 - alpha) * data['speed'][0]

torque_pred_wt = out_trq_wt[0] * 120
torque_pred_wt = alpha * torque_pred_wt + (1 - alpha) * data['torque'][0]

alpha = 0.0331
speed_pred_ekf = out_spd_ekf[0] * 80
speed_pred_ekf = alpha * speed_pred_ekf + (1 - alpha) * data['speed'][0]

torque_pred_ekf = out_trq_ekf[0] * 120
torque_pred_ekf = alpha * torque_pred_ekf + (1 - alpha) * data['torque'][0]

alpha = 0.0345
speed_pred_tv = out_spd_tv[0] * 80
speed_pred_tv = alpha * speed_pred_tv + (1 - alpha) * data['speed'][0]

torque_pred_tv = out_trq_tv[0] * 120
torque_pred_tv = alpha * torque_pred_tv + (1 - alpha) * data['torque'][0]



In [6]:
data['speed_CASE_A'] = speed_pred
data['torque_CASE_A'] = torque_pred

data['speed_CASE_B'] = speed_pred_b
data['torque_CASE_B'] = torque_pred_b

data['speed_CASE_C'] = speed_pred_c
data['torque_CASE_C'] = torque_pred_c

data['speed_CASE_A'] = speed_pred
data['torque_CASE_A'] = torque_pred

data['speed_EKF'] = speed_pred_ekf
data['torque_EKF'] = torque_pred_ekf

data['speed_WT'] = speed_pred_wt
data['torque_WT'] = torque_pred_wt

data['speed_MCTV'] = speed_pred_tv
data['torque_MCTV'] = torque_pred_tv

data['speed_DAE'] = speed_pred_da
data['torque_DAE'] = torque_pred_da

data['speed_MD'] = speed_pred_md
data['torque_MD'] = torque_pred_md

savemat('quasi_static1.mat', data)

In [7]:
st = int(5 / 0.004)
et = int(55 / 0.004)
print ('CASE A', np.max(np.abs(data['speed'][0][st:et] - speed_pred[st:et])))
print ('CASE B', np.max(np.abs(data['speed'][0][st:et] - speed_pred_b[st:et])))
print ('CASE C', np.max(np.abs(data['speed'][0][st:et] - speed_pred_c[st:et])))
print ('EKF', np.max(np.abs(data['speed'][0][st:et] - speed_pred_ekf[st:et])))
print ('WT', np.max(np.abs(data['speed'][0][st:et] - speed_pred_wt[st:et])))
print ('MCTV', np.max(np.abs(data['speed'][0][st:et] - speed_pred_tv[st:et])))
print ('DAE', np.max(np.abs(data['speed'][0][st:et] - speed_pred_da[st:et])))
print ('MD', np.max(np.abs(data['speed'][0][st:et] - speed_pred_md[st:et])))

CASE A 0.19808661489314972
CASE B 0.21089272911868306
CASE C 0.2193782729585223
EKF 0.19561003300078283
WT 0.19528877867394812
MCTV 0.19689783482529322
DAE 0.19517771914275528
MD 0.19459335471418626
